In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import plotly.express as px
import numpy as np
import os
from xgboost import XGBClassifier

# SEM STANDARD SCALER

In [5]:
# df_original = pd.read_csv('/home/enacom/Downloads/finep_alz/data/prep_balanced_data_normalized.csv')
# columns_to_exclude = ['Subject', 'Session', "GENDER"]

# df = df_original.drop(columns= columns_to_exclude)
# df['Age'] = df['Age'].astype(str).str.split('.').str[0]
# df['Age'] = df['Age'].astype(float)

# df.dropna(axis = 0, inplace = True)

In [6]:
# df_original = pd.read_csv('/home/enacom/Downloads/finep_alz/data/prep_balanced_data_normalized.csv')
# df = df_original[["AD", "ctx-lh-entorhinal", "ctx-lh-parahippocampal", "Left-Accumbens-area", "Left-Hippocampus", "Right-Accumbens-area", "ctx-rh-entorhinal", "ctx-rh-parahippocampal", "hpcr_wm_left",	"hpcr_wm_right", "Right_choroid_volume", "Right-Hippocampus", "CSF", "Age"]]

# # columns_to_exclude = ['Subject', 'Session', "GENDER"]

# # df = df_original.drop(columns= columns_to_exclude)
# df['Age'] = df['Age'].astype(str).str.split('.').str[0]
# # df["Age"].astype(int)
# df.dropna(axis = 0, inplace = True)
# df.head()

In [7]:
# X = df.drop(columns=["AD"], axis=1)
# y = df["AD"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
train_df = pd.read_csv("/home/enacom/Downloads/finep_alz/data/dataset_hani_train.csv")
test_df = pd.read_csv("/home/enacom/Downloads/finep_alz/data/dataset_hani_test.csv")


columns_to_exclude = ['Subject', 'Session', "GENDER"]
train_df.drop(columns=columns_to_exclude, inplace= True)
test_df.drop(columns=columns_to_exclude, inplace= True)
# df['Age'] = df['Age'].astype(str).str.split('.').str[0]
# df["Age"].astype(int)
train_df.dropna(axis = 0, inplace = True)
test_df.dropna(axis = 0, inplace = True)
print(test_df.shape)
print(train_df.shape)
X_train = train_df.drop('AD', axis=1)
y_train = train_df['AD']
X_test = test_df.drop('AD', axis=1)
# Assuming the test set also has the target column for evaluation
y_test = test_df['AD']

(94, 93)
(415, 93)


In [9]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, recall_score, precision_score, accuracy_score


def recall_youden(y_true, y_proba):
    fpr, tpr, thrs = roc_curve(y_true, y_proba)
    j = tpr - fpr
    idx = np.argmax(j)
    best_thr = thrs[idx]
    
    return recall_score(y_true, y_proba>=best_thr)   

def precision_youden(y_true, y_proba):
    fpr, tpr, thrs = roc_curve(y_true, y_proba)
    j = tpr - fpr
    idx = np.argmax(j)
    best_thr = thrs[idx]
    
    return precision_score(y_true, y_proba>=best_thr)  

from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, recall_score, precision_score, accuracy_score


def recall_youden(y_true, y_proba):
    fpr, tpr, thrs = roc_curve(y_true, y_proba)
    j = tpr - fpr
    idx = np.argmax(j)
    best_thr = thrs[idx]
    
    return recall_score(y_true, y_proba>=best_thr)   

def precision_youden(y_true, y_proba):
    fpr, tpr, thrs = roc_curve(y_true, y_proba)
    j = tpr - fpr
    idx = np.argmax(j)
    best_thr = thrs[idx]
    
    return precision_score(y_true, y_proba>=best_thr)  

def accuracy_youden(y_true, y_proba):
    # Calculate the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    
    # Calculate Youden's J statistic
    J = tpr - fpr
    # Find the optimal threshold
    idx = np.argmax(J)
    best_thresh = thresholds[idx]
    
    # Make predictions based on the optimal threshold
    y_pred = (y_proba >= best_thresh).astype(int)
    
    # Calculate accuracy
    return accuracy_score(y_true, y_pred)

recall_youden_scorer = make_scorer(recall_youden, needs_proba=True)
precision_youden_scorer = make_scorer(precision_youden, needs_proba=True)
accuracy_youden_scorer = make_scorer(accuracy_youden, needs_proba=True)

/home/enacom/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [10]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    # Variables
    'max_depth': [2, 4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],

    # Fixed parameters
    # 'scale_pos_weight': [1.48],
    
    'random_state': [42],

    # Additional parameter
    'enable_categorical': [True]  # Adding the enable_categorical flag
}

results = []
for param in list(ParameterGrid(param_grid)):
    # param['enable_categorical'] = True 
    cf = XGBClassifier(**param)
    cf.fit(X_train, y_train)
    y_pred = cf.predict_proba(X_test)
    r = {
        'recall_youden':recall_youden(y_test, y_pred[:, 1]),
        'precision_youden':precision_youden(y_test, y_pred[:, 1]),
        'accuracy_youden':accuracy_youden(y_test, y_pred[:, 1]),
    }
    results.append(r)
 
    
results  

[{'recall_youden': 0.7450980392156863,
  'precision_youden': 0.5846153846153846,
  'accuracy_youden': 0.574468085106383},
 {'recall_youden': 0.19607843137254902,
  'precision_youden': 0.7692307692307693,
  'accuracy_youden': 0.5319148936170213},
 {'recall_youden': 0.19607843137254902,
  'precision_youden': 0.8333333333333334,
  'accuracy_youden': 0.5425531914893617},
 {'recall_youden': 0.21568627450980393,
  'precision_youden': 0.7333333333333333,
  'accuracy_youden': 0.5319148936170213},
 {'recall_youden': 0.11764705882352941,
  'precision_youden': 1.0,
  'accuracy_youden': 0.5212765957446809},
 {'recall_youden': 0.1568627450980392,
  'precision_youden': 1.0,
  'accuracy_youden': 0.5425531914893617},
 {'recall_youden': 0.23529411764705882,
  'precision_youden': 0.75,
  'accuracy_youden': 0.5425531914893617},
 {'recall_youden': 0.23529411764705882,
  'precision_youden': 0.8,
  'accuracy_youden': 0.5531914893617021},
 {'recall_youden': 0.21568627450980393,
  'precision_youden': 0.785714

In [11]:
cv_results = results

data = {
    'Mean Test Accuracy': [result['accuracy_youden'] for result in cv_results],
    'Mean Test Precision': [result['precision_youden'] for result in cv_results],
    'Parameters': [str(params) for params in list(ParameterGrid(param_grid))]
}

import plotly.express as px

fig = px.scatter(
    data,
    x='Mean Test Accuracy',
    y='Mean Test Precision',
    hover_data=['Parameters'],
    title='Precision vs. Accuracy with Parameter Details',
    labels={'Mean Test Accuracy': 'Accuracy', 'Mean Test Precision': 'Precision'}
)

fig.show()

In [12]:
def identify_pareto(scores):
    pareto_index = []
    for i, (c1, c2) in enumerate(scores):
        dominated = any((c1 < other_c1 and c2 < other_c2) for j, (other_c1, other_c2) in enumerate(scores) if i != j)
        if not dominated:
            pareto_index.append(i)
    return pareto_index

recall = [result['recall_youden'] for result in cv_results]
precision = [result['precision_youden'] for result in cv_results]

scores = list(zip(recall, precision))

pareto_indices = identify_pareto(scores)

pareto_data = {
    'Mean Test Recall': [recall[i] for i in pareto_indices],
    'Mean Test Precision': [precision[i] for i in pareto_indices],
    'Parameters': [str(list(ParameterGrid(param_grid))[i]) for i in pareto_indices]
}

import plotly.express as px

pareto_fig = px.scatter(
    pareto_data,
    x='Mean Test Recall',
    y='Mean Test Precision',
    hover_data=['Parameters'],
    title='Pareto Front of Precision vs. Recall',
    labels={'Mean Test Recall': 'Recall', 'Mean Test Precision': 'Precision'}
)

pareto_fig.show()

# COM STANDARD SCALER

In [13]:
scaler = StandardScaler()

# Fit on training data
scaler.fit(X_train)

# Transform training and testing data
X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)

NameError: name 'X' is not defined

In [ ]:
results = []
for param in list(ParameterGrid(param_grid)):
    cf = XGBClassifier(**param)
    cf.fit(X_train, y_train)
    y_pred = cf.predict_proba(X_test)
    r = {
        'recall_youden':recall_youden(y_test, y_pred[:, 1]),
        'precision_youden':precision_youden(y_test, y_pred[:, 1])
    }
    results.append(r)
 
    
results  

In [ ]:
cv_results = results

data = {
    'Mean Test Recall': [result['recall_youden'] for result in cv_results],
    'Mean Test Precision': [result['precision_youden'] for result in cv_results],
    'Parameters': [str(params) for params in list(ParameterGrid(param_grid))]
}

import plotly.express as px

fig = px.scatter(
    data,
    x='Mean Test Recall',
    y='Mean Test Precision',
    hover_data=['Parameters'],
    title='Precision vs. Recall with Parameter Details',
    labels={'Mean Test Recall': 'Recall', 'Mean Test Precision': 'Precision'}
)

fig.show()

In [ ]:
def identify_pareto(scores):
    pareto_index = []
    for i, (c1, c2) in enumerate(scores):
        dominated = any((c1 < other_c1 and c2 < other_c2) for j, (other_c1, other_c2) in enumerate(scores) if i != j)
        if not dominated:
            pareto_index.append(i)
    return pareto_index

recall = [result['recall_youden'] for result in cv_results]
precision = [result['precision_youden'] for result in cv_results]

scores = list(zip(recall, precision))

pareto_indices = identify_pareto(scores)

pareto_data = {
    'Mean Test Recall': [recall[i] for i in pareto_indices],
    'Mean Test Precision': [precision[i] for i in pareto_indices],
    'Parameters': [str(list(ParameterGrid(param_grid))[i]) for i in pareto_indices]
}

import plotly.express as px

pareto_fig = px.scatter(
    pareto_data,
    x='Mean Test Recall',
    y='Mean Test Precision',
    hover_data=['Parameters'],
    title='Pareto Front of Precision vs. Recall',
    labels={'Mean Test Recall': 'Recall', 'Mean Test Precision': 'Precision'}
)

pareto_fig.show()